In [ ]:
import networkx as nx
from collections import defaultdict
from itertools import combinations
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import sys
import dimod 
import dwave_networkx as dnx
import metis
####
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import neal
import dwave.inspector

hed=list(range(0,26))

df=pd.read_csv("./metis_graph_weights_002.dat",skiprows=[0],header=None,sep=" ")
dh=pd.read_csv("./metis_graph_simple_002.dat",skiprows=[0],header=None,sep=" ")
dtt=pd.read_csv("./metis_graph_std_002.dat",skiprows=[0],header=None,sep=" ")

In [ ]:
#Normalise Data
max_value1 = df.max(skipna=True).max()
df=df/max_value1

## Map Metis output to graph suitable for QA

In [ ]:
#df--->Weight list
#dh--->Link list

# [0,....]
# [1,....]
# [2,....]
d=[]
e=[]
w=[]
d_f=[]
e_f=[]
for i in range(0,27):
    for j in range(1,27):
        d.append(i)
        e.append(dh.iloc[i,j])
        w.append(df.iloc[i,j+1])

gp=pd.DataFrame()
gp["s"]=d
gp["t"]=e
gp["w"]=w
gp["t"]=gp["t"]-1
gp=gp[gp["s"]<gp["t"]] 


G=nx.from_pandas_edgelist(gp,source="s",target="t")

nx.draw(G, with_labels=True)

#nx.degree(G)
nx.is_weighted(G) #i.e. not weighted yet




In [ ]:
###This manually adds the correct weight to each edge combination

for y,z in G.edges:
    #a=gp[(gp["s"]==y) & (gp["t"]==z)].index.values
    a=gp[( (gp["s"]==y) & (gp["t"]==z) ) | ( (gp["s"]==z) & (gp["t"]==y) ) ].index.values  #Not unique 
   # ll=gp["w"].loc[a].values            #original
    ll = gp["w"].loc[a].values.flatten() #try debug for adjacency matrix
    G[y][z]["weight"]=ll

In [ ]:
#manually add node weights
node_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
weights = df[1].tolist()

for node, weight in zip(node_ids, weights):
    if G.has_node(node):
        G.nodes[node]['weight'] = weight

In [ ]:
nx.is_weighted(G) #debug check


## MEtis Cut 

In [ ]:
#Metis cut
%time
(edgecuts, parts) = metis.part_graph(G, 2,)

cut_weight = 0
####Cut weights
for u, v, data in G.edges(data=True):
    if parts[u] != parts[v]:
        cut_weight += data['weight']

print('cut edges',cut_weight)

####Partition weights
partition_weights = defaultdict(int)

for node, data in G.nodes(data=True):
    partition_label = parts[node]
    node_weight = data.get('weight',)  
    partition_weights[partition_label] += node_weight

print('partition weights',partition_weights)
metis_node= abs(partition_weights[1]-partition_weights[0])/(0.5*(partition_weights[0]+partition_weights[1]))
metis_edge=cut_weight

In [ ]:
#PLot Graph STructure
node_colors = []
for node in G.nodes():
    if parts[node] == 0:
        node_colors.append('#1f78b4')
    else:
        node_colors.append('#e31a1c')

# Draw the graph
pos = nx.circular_layout(G)  # positions for all nodes; you can use other layouts as well
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=500)
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10)

# Highlight the cut edges 
cut_edge_list = [(u, v) for u, v, data in G.edges(data=True) if parts[u] != parts[v]]
nx.draw_networkx_edges(G, pos, edgelist=cut_edge_list, width=2, alpha=0.5, edge_color="r", style="dashed")

plt.title("Graph Partition Visualization (METIS)")
plt.axis('off')  # Turn off the axis

#plt.savefig("gen1.pdf")

In [ ]:
                      ##Can Run this block after QA to get same plot as above but using QA instead of Metis 
#import matplotlib.pyplot as plt
#import networkx as nx
#import numpy as np

## Sort nodes by their label or any other criterion you have
#sorted_nodes = sorted(G.nodes(), key=lambda x: x)  # Adjust if your sorting criterion is different

## Calculate positions - starting with node 0 at the top
#n = len(sorted_nodes)
#pos = {node: (np.cos(2 * np.pi * i / n + np.pi/2), np.sin(2 * np.pi * i / n + np.pi/2)) for i, node in enumerate(sorted_nodes)}

## Define node colors based on partitions
#node_colors = ['#1f78b4' if parts[node] == 0 else '#e31a1c' for node in sorted_nodes]

## Draw the graph nodes, edges, and labels using the calculated positions
#nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=500)
#nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
#nx.draw_networkx_labels(G, pos, font_size=10)

## Highlight the cut edges
#cut_edge_list = [(u, v) for u, v in G.edges() if parts[u] != parts[v]]
#nx.draw_networkx_edges(G, pos, edgelist=cut_edge_list, width=2, alpha=0.5, edge_color="r", style="dashed")

##plt.title("Graph Partition Visualization")
#plt.axis('off')  
#plt.tight_layout
#plt.savefig("gen1.pdf", format='pdf', bbox_inches='tight', pad_inches=0.2)


## QA Cuts

In [ ]:
# ------- Unweighted Cut - Single Run (Debugging)-------
num_reads = 1000
gamma = 80

Q = defaultdict(int)

# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1
    Q[(v,v)] += 1
    Q[(u,v)] += -2

for i in G.nodes:
    Q[(i,i)] += gamma*(1-len(G.nodes))

for i, j in combinations(G.nodes, 2):
	Q[(i,j)] += 2*gamma

# ------- Run 

# Set chain strength
chain_strength = gamma*len(G.nodes)


sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chain_strength,
                               num_reads=num_reads,
                               label='GP')

#Isolate best solution
sample = response.record.sample[0]

# In the case when n is odd, the set may have one more or one fewer nodes
if sum(sample) in [math.floor(len(G.nodes)/2), math.ceil(len(G.nodes)/2)]:
    num_cut_edges = 0
    for u, v in G.edges:
        num_cut_edges += sample[u] + sample[v] - 2*sample[u]*sample[v]
    print("Valid partition found with", num_cut_edges, "cut edges.")
else:

    print("Invalid partition.")

In [ ]:
# ------- Weighted - Single run (Debugging)-------
num_reads = 1000
gamma = 1

Q = defaultdict(int)

# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1 * G[u][v]["weight"]
    Q[(v,v)] += 1 * G[u][v]["weight"]
    Q[(u,v)] += -2* G[u][v]["weight"]

#for i in G.nodes:
#    Q[(i,i)] += gamma*(1-len(G.nodes))*df.iloc[i,1]

#for i, j in combinations(G.nodes, 2):
#    Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

for i in G.nodes:
    Q[(i,i)] += gamma*(df.iloc[i,1]-df[1].sum())*df.iloc[i,1]

for i, j in combinations(G.nodes, 2):
    Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

# ------- Run QUBO

# Set chain strength
chain_strength = 100*len(G.nodes)
#chain_strength = g*len(G.nodes)
# Run the QUBO
sampler = EmbeddingComposite(DWaveSampler(token='')) #add token
response = sampler.sample_qubo(Q,
                               chain_strength=chain_strength,
                               num_reads=num_reads,
                               label= 'Graph Partitioning')


# See if the best solution found is feasible, and if so print the number of cut edges.
sample = response.record.sample[0]

# In the case when n is odd, the set may have one more or one fewer nodes

num_cut_edges = 0
for u, v in G.edges:
      num_cut_edges += sample[u] + sample[v] - 2*sample[u]*sample[v]
print("Valid partition found with", num_cut_edges, "cut edges.")
total_cut_weight = 0
for u, v in G.edges:
    if sample[u] != sample[v]:  # If the two nodes of the edge are in different partitions
          total_cut_weight += G[u][v]["weight"]  # Add the weight of the edge
print("Valid partition found with cut edges of total weight:", total_cut_weight)

#dwave.inspector.show(response)       
d_output=response.to_pandas_dataframe()    



In [ ]:
#Running weighted cut  multiple times for averages
from collections import defaultdict
from itertools import combinations
import networkx as nx
from dwave.system import DWaveSampler, EmbeddingComposite
import pandas as pd

# Initialize variables for averaging
total_cut_weight_accumulated = 0
total_weight_partition1_accumulated = 0
total_weight_partition2_accumulated = 0
num_iterations = 5

# Run the code 5 times
for _ in range(num_iterations):
    num_reads = 1000
    gamma = 1

    Q = defaultdict(int)

    # Fill in Q matrix
    # Fill in Q matrix
    for u, v in G.edges:
        Q[(u,u)] += 1 * G[u][v]["weight"]
        Q[(v,v)] += 1 * G[u][v]["weight"]
        Q[(u,v)] += -2* G[u][v]["weight"]
    for i in G.nodes:
        Q[(i,i)] += gamma*(df.iloc[i,1]-df[1].sum())*df.iloc[i,1]

    for i, j in combinations(G.nodes, 2):
        Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

    # Set chain strength and run the QUBO on the solver
    chain_strength = 100 * len(G.nodes)
    sampler = EmbeddingComposite(DWaveSampler(token='')) # Add your token 
    response = sampler.sample_qubo(Q, chain_strength=chain_strength, num_reads=num_reads)

    sample = response.record.sample[0]

    # Calculate cut edges and total cut weight
    total_cut_weight = 0
    for u, v in G.edges:
        if sample[u] != sample[v]:
            total_cut_weight += G[u][v]["weight"]

    # Accumulate total cut weight
    total_cut_weight_accumulated += total_cut_weight

    # Calculate and accumulate partition weights
    partition1_weight = sum(G.nodes[n]['weight'] for n in G if sample[n] == 0)
    partition2_weight = sum(G.nodes[n]['weight'] for n in G if sample[n] == 1)
    total_weight_partition1_accumulated += partition1_weight
    total_weight_partition2_accumulated += partition2_weight

# Calculate averages
average_cut_weight = total_cut_weight_accumulated / num_iterations
average_partition1_weight = total_weight_partition1_accumulated / num_iterations
average_partition2_weight = total_weight_partition2_accumulated / num_iterations

print("Average cut edges weight:", average_cut_weight)
print("Average weight of partition 1:", average_partition1_weight)
print("Average weight of partition 2:", average_partition2_weight)



In [ ]:
#Ignore (sanity check)
#a1=9.700231
#a2=10.268669
#abs(a1-a2)/(0.5*(a1+a2))

In [ ]:
#Check size of partitions
# Initialize partition weights
partition_0_weight = 0
partition_1_weight = 0

# Calculate partition weights
for node in G.nodes:
    if sample[node] == 0:
        partition_0_weight += G.nodes[node]['weight']
    else:
        partition_1_weight += G.nodes[node]['weight']

print("Weight of partition 0:", partition_0_weight)
print("Weight of partition 1:", partition_1_weight)
#qa_node=abs(partition_0_weight-partition_1_weight) / (0.5*(partition_0_weight+partition_1_weight))
#qa_node

In [ ]:
#!pip install seaborn
# Create a matrix to store the weights of cut edges
matrix = np.zeros((len(G.nodes()), len(G.nodes())))
count_qa=0
for u, v, data in G.edges(data=True):
    if sample[u] != sample[v]:  # Check if the edge is a cut edge
        matrix[u-1][v-1] = data['weight']
        matrix[v-1][u-1] = data['weight']  
        count_qa+=1

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, cmap='viridis')
#plt.savefig('heat_qa.pdf')
plt.show()
count_qa #check number of cuts

In [ ]:
#METIS
# Create a matrix to store the weights of cut edges
matrix = np.zeros((len(G.nodes()), len(G.nodes())))
count_metis=0
for u, v, data in G.edges(data=True):
    if parts[u] != parts[v]:  # Check if the edge is a cut edge
        matrix[u-1][v-1] = data['weight']
        matrix[v-1][u-1] = data['weight'] 
        count_metis+=1
# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, cmap='viridis')
#plt.savefig('heat_metis.pdf')
plt.show()
count_metis #check number of cuts

In [ ]:
# ------- Weighted to find cbf change with gamma -------

num_reads = 1000
gammas = [0.1, 0.5, 1, 10, 100,500]  # adjust this list as needed
mean_chain_break_fractions = []
num_runs_per_gamma = 5

for gamma in gammas:
    chain_break_fractions_for_this_gamma = []

    for _ in range(num_runs_per_gamma):
        Q = defaultdict(int)

        # Fill in Q matrix
        for u, v in G.edges:
            Q[(u, u)] += 1 * G[u][v]["weight"]
            Q[(v, v)] += 1 * G[u][v]["weight"]
            Q[(u, v)] += -2 * G[u][v]["weight"]

        for i in G.nodes:
            Q[(i, i)] += gamma * (df.iloc[i, 1] - df[1].sum()) * df.iloc[i, 1]

        for i, j in combinations(G.nodes, 2):
            Q[(i, j)] += 2 * gamma * df.iloc[i, 1] * df.iloc[j, 1]

        # Set chain strength
        #chain_strength = 100 * len(G.nodes)

        # Run the QUBO on the solver
        sampler = EmbeddingComposite(DWaveSampler(token='')) # Add your token
        #response = sampler.sample_qubo(Q, chain_strength=chain_strength, num_reads=num_reads, label='Graph Partitioning')
        response = sampler.sample_qubo(Q, num_reads=num_reads, label='Graph Partitioning')
        # Calculate mean chain break fraction for this run
        mean_chain_break_fraction_for_this_run = sum(response.record.chain_break_fraction) / len(response.record.chain_break_fraction)
        chain_break_fractions_for_this_gamma.append(mean_chain_break_fraction_for_this_run)

    # Calculate the average chain break fraction for this gamma over all runs
    avg_chain_break_fraction = sum(chain_break_fractions_for_this_gamma) / num_runs_per_gamma
    mean_chain_break_fractions.append(avg_chain_break_fraction)

    print(f"For gamma={gamma}, average chain break fraction over {num_runs_per_gamma} runs: {avg_chain_break_fraction}")

# Plot gamma values against mean chain break fractions
plt.plot(gammas, mean_chain_break_fractions, '-o')
plt.xlabel('Lagrange Parameter')
plt.ylabel('Chain Break Fraction')
#plt.title(f'Average Chain Break Fraction vs. Gamma (over {num_runs_per_gamma} runs)')
#plt.grid(True)
#plt.savefig('cbf_vs_gamma_s1.pdf')

## Pareto Front 

In [ ]:
# ------- Weighted Pareto for single gamma-------
num_reads = 1000
gamma = 10000

Q = defaultdict(int)

# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1 * G[u][v]["weight"]
    Q[(v,v)] += 1 * G[u][v]["weight"]
    Q[(u,v)] += -2* G[u][v]["weight"]

#for i in G.nodes:
#    Q[(i,i)] += gamma*(1-len(G.nodes))*df.iloc[i,1]

#for i, j in combinations(G.nodes, 2):
#    Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

for i in G.nodes:
    Q[(i,i)] += gamma*(df.iloc[i,1]-df[1].sum())*df.iloc[i,1]

for i, j in combinations(G.nodes, 2):
    Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

# ------- Run QUBO

# Set chain strength
chain_strength = gamma*len(G.nodes)

# Run the QUBO on the solver from your config file
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chain_strength,
                               num_reads=num_reads,
                               label= 'Graph Partitioning')

# See if the best solution found is feasible, and if so print the number of cut edges.
f1=[]
f2=[]
for kk in range(0,len(response.record)):
    sample = response.record.sample[kk]
    fx2 = 0
    for u, v in G.edges:
        fx2 += (sample[u] + sample[v] - 2*sample[u]*sample[v])*G[u][v]["weight"][0]
    f2.append(fx2)  
    out_n=[0,26,24,25,20,18,19,23,21,22,8,6,7,2,1,5,3,4,17,15,16,11,9,10,14,12,13]
    final=out_n
    for i in range(0,27):
        final[i]=sample[i]*df.iloc[out_n[i],1]
    fx1=abs(sum(final)-df[1].sum())
    f1.append(fx1)
#dwave.inspector.show(response)       
d_output=response.to_pandas_dataframe()    

In [ ]:
# ------- Weighted Pareto for looped gamma-------
num_reads = 1000
gamma = 10000
#gamma_range=np.arange(100,5000,100) #Experimenting with different ranges
#gamma_range=np.arange(0.05,20,0.2)
gamma_range=np.arange(0.05,50,0.5)
f1=[]
f2=[]
for gamma in gamma_range: 
    print(gamma)
    Q = defaultdict(int)

    # Fill in Q matrix
    for u, v in G.edges:
        Q[(u,u)] += 1 * G[u][v]["weight"]
        Q[(v,v)] += 1 * G[u][v]["weight"]
        Q[(u,v)] += -2* G[u][v]["weight"]

#for i in G.nodes:
#    Q[(i,i)] += gamma*(1-len(G.nodes))*df.iloc[i,1]

#for i, j in combinations(G.nodes, 2):
#    Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

    for i in G.nodes:
        Q[(i,i)] += gamma*(df.iloc[i,1]-df[1].sum())*df.iloc[i,1]

    for i, j in combinations(G.nodes, 2):
        Q[(i,j)] += 2*gamma*df.iloc[i,1]*df.iloc[j,1]

# ------- Run our QUBO on the QPU -------

# Set chain strength
    chain_strength = gamma*len(G.nodes)

# Run the QUBO 
    sampler = EmbeddingComposite(DWaveSampler(token='')) #Add your token
    response = sampler.sample_qubo(Q,
                                   chain_strength=chain_strength,
                                   num_reads=num_reads,
                                   label= 'Graph Partitioning')

# See if the best solution found is feasible, and if so print the number of cut edges.
    for kk in range(0,len(response.record)):  #Loop Through samples
        sample = response.record.sample[kk]
        fx2 = 0
        for u, v in G.edges:
            fx2 += (sample[u] + sample[v] - 2*sample[u]*sample[v])*G[u][v]["weight"][0] #Find weight of cuts
        f2.append(fx2)                                                                  #Store
        out_n=[0,26,24,25,20,18,19,23,21,22,8,6,7,2,1,5,3,4,17,15,16,11,9,10,14,12,13]  #Find weight of partition
        final=out_n
        for i in range(0,27):
            final[i]=sample[i]*df.iloc[out_n[i],1]
        #fx1=abs(sum(final)-df[1].sum())
        fx1=abs(sum(final)-abs(sum(final)-df[1].sum()))/(0.5*df[1].sum())
        f1.append(fx1)                                                                  #Store
#dwave.inspector.show(response)       
    d_output=response.to_pandas_dataframe()    

In [ ]:
#Debug Check
#df[1].sum()
#abs(sum(final)-df[1].sum())
#sum(final)

In [ ]:
#Debug check
d_output=response.to_pandas_dataframe()  
d_output
plt.scatter(f1,f2)
d_output
len(f1)

In [ ]:
df_candidates = pd.DataFrame(candidate_solutions, columns=['Imbalance', 'Cut Edge Weights'])
# Save DataFrames to CSV
df_candidates.to_csv('candidate_solutions.csv', index=False)

In [ ]:
#Evaluate pareto front from f1 and f2 candidate solutions
def is_dominated(solution_a, solution_b):
    """
    Check if solution_a is dominated by solution_b.
    """
    a_obj1, a_obj2 = solution_a  # Objective values of solution_a
    b_obj1, b_obj2 = solution_b  # Objective values of solution_b

    return a_obj1 >= b_obj1 and a_obj2 >= b_obj2 and (a_obj1 > b_obj1 or a_obj2 > b_obj2)

def find_pareto_front(candidate_solutions):
    """
    Find the Pareto front from a list of candidate solutions.
    """
    pareto_front = []
    for solution in candidate_solutions:
        is_dominated_by_others = False
        for other_solution in candidate_solutions:
            if solution != other_solution and is_dominated(solution, other_solution):
                is_dominated_by_others = True
                break
        if not is_dominated_by_others:
            pareto_front.append(solution)
    return pareto_front
candidate_solutions=list(zip(f1, f2))
pareto_front = find_pareto_front(candidate_solutions)
print("Pareto Front:")
#for solution in pareto_front:
#    print(solution)
#candidate_solutions
x_values = [solution[0] for solution in pareto_front]
y_values = [solution[1] for solution in pareto_front]


fig=plt.figure()
plt.scatter(x_values, y_values,label="QA")
plt.scatter(0.189,5.2,label="METIS")
#plt.title("Pareto Front")
plt.xlabel("Solution Imbalance")
plt.ylabel("Cut Edge Weights")
plt.legend()
#plt.savefig("pareto_0.1_10.pdf", bbox_inches="tight")


In [ ]:
import matplotlib.pyplot as plt

# Assuming candidate_solutions, is_dominated, find_pareto_front functions, and METIS solution are defined

# Find the Pareto front
pareto_front = find_pareto_front(candidate_solutions)

# Solutions that dominate the METIS solution, excluding those on the Pareto front
dominates_metis = [sol for sol in candidate_solutions if is_dominated(metis_solution, sol) and sol not in pareto_front]

# Sorting Pareto front points for line plot
pareto_front_sorted = sorted(pareto_front, key=lambda x: x[0])  # Sort by the first objective (e.g., imbalance)


fig, ax = plt.subplots()

# Pareto front points
ax.scatter([sol[0] for sol in pareto_front], [sol[1] for sol in pareto_front], label='Pareto Front',zorder=5)
# Connect Pareto front points with a line
ax.plot([sol[0] for sol in pareto_front_sorted], [sol[1] for sol in pareto_front_sorted],linestyle="--",zorder=4)

# Solutions dominating METIS
ax.scatter([sol[0] for sol in dominates_metis], [sol[1] for sol in dominates_metis], color='lightblue', label='Dominates METIS',marker=".")

# METIS solution
ax.scatter(*metis_solution, color='red', label='METIS', zorder=5)  # zorder for drawing order

ax.set_xlim(left=-0.02, right=0.45)

ax.legend()
plt.rcParams.update({'font.size': 14})
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel("Solution Imbalance")
plt.ylabel("Cut Edge Weights")
plt
plt.tight_layout
plt.savefig("pareto_f2.pdf")

In [ ]:
#Same plot as above but includes all candidate solutions in light grey
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

fig, ax = plt.subplots()

# Plot all candidate solutions lightly for context
candidates_plot = ax.scatter([sol[0] for sol in candidate_solutions], 
                             [sol[1] for sol in candidate_solutions], 
                             color='gray', alpha=0.01, marker="x")

# Pareto front points
pareto_front_plot = ax.scatter([sol[0] for sol in pareto_front], 
                               [sol[1] for sol in pareto_front], 
                               label='Pareto Front', zorder=5)

# Connect Pareto front points with a line
ax.plot([sol[0] for sol in pareto_front_sorted], 
        [sol[1] for sol in pareto_front_sorted], 
        linestyle="--", zorder=4)

# Solutions dominating METIS (explicitly excluding Pareto front)
dominates_metis_plot = ax.scatter([sol[0] for sol in dominates_metis], 
                                  [sol[1] for sol in dominates_metis], 
                                  color='lightblue', label='Dominates METIS', marker=".")

# METIS solution
metis_plot = ax.scatter(*metis_solution, color='red', label='METIS', zorder=5)  # zorder for drawing order

# Create custom legend handles
candidates_legend = mlines.Line2D([], [], color='gray', marker='x', linestyle='None',
                                  markersize=8, label='Candidates',alpha=0.5)
pareto_legend = mlines.Line2D([], [], marker='o', linestyle='None',
                              markersize=10, label='Pareto Front')
dominates_metis_legend = mlines.Line2D([], [], color='lightblue', marker='.', linestyle='None',
                                        markersize=10, label='Dominates METIS')
metis_legend = mlines.Line2D([], [], color='red', marker='o', linestyle='None',
                              markersize=10, label='METIS')


ax.legend(handles=[candidates_legend, pareto_legend, dominates_metis_legend, metis_legend])

ax.set_xlim(left=-0.02, right=0.45)
ax.set_ylim(bottom=2.5, top=6.0)
plt.xlabel("Solution Quality")
plt.ylabel("Cut Edge Weights")
plt.tight_layout()



#plt.savefig("pareto_f5.pdf")


In [ ]:
len(dominates_metis)